<a href="https://colab.research.google.com/github/nauam/UnB_MAR/blob/main/MAR_Code_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
from numpy import sinc, pi, sin, diag, matmul as mult, array, zeros, ones, eye, sqrt
from numpy.linalg  import inv
from scipy.special import spherical_jn, spherical_yn

In [277]:
def F_Q(u,n,theta):
  return theta/2*(sinc((u-n)*theta/pi) + sinc((u+n+1)*theta/pi))

def F_S(u,n,theta):
  S_mat = []
  for i_ in range(0, len(theta)):
    if theta[i_][i_] == 0:
        S_mat.append(0)
    else:
        S_mat.append(2/pi*(M__Q(u,n,theta[i_][i_]) - M__Q(u,0,theta[i_][i_])*M__Q(0,n,theta[i_][i_])/M__Q(0,0,theta[i_][i_])))
  return diag(S_mat)

def F_SphericalFunction(z,n):
  N = len(z)
  Z1 = zeros((N,N,n),dtype = 'complex_');  Z3 = zeros((N,N,n),dtype = 'complex_');  K1 = zeros((N,N,n),dtype = 'complex_');  K3 = zeros((N,N,n),dtype = 'complex_')
  n_ =  list(range(1,n+1))
  for i_ in range(0, N):
    Z1[i_][i_][:] = spherical_jn(n_, z[i_][i_], derivative=False)
    Z2            = spherical_yn(n_, z[i_][i_], derivative=False)
    Z3[i_][i_][:] = Z1[i_][i_][:] + 1j*Z2;

    K1[i_][i_][:] = spherical_jn(n_, z[i_][i_], derivative=True) + Z1[i_][i_][:]/z[i_][i_]
    K2            = spherical_yn(n_, z[i_][i_], derivative=True) + Z2           /z[i_][i_]
    K3[i_][i_][:] = K1[i_][i_][:] + 1j*K2;

  return Z1, Z3, K1, K3

def S_ProbeFedA(k0r, Ang1, Ang2):
  frq = 9*10**9
  c0  = 299792458
  k0  = 2*pi*frq/c0

  r    = diag([           1.000,    0.970])*k0r/k0
  th_r = diag([           Ang1 ,     Ang2])*pi/180
  ep_r = diag([  1.000,   1.300,    1.000])
  mu_r = diag([  1.000,   1.000,    1.000])

  return r, th_r, ep_r, mu_r, k0

def S_ProbeFedB():
  frq = 9*10**9
  c0  = 299792458
  k0  = 2*pi*frq/c0

  r    = diag([           0.038,   0.035])
  th_r = diag([          16.000, 160.000])*pi/180
  ep_r = diag([  1.000,   1.230,   1.000])
  mu_r = diag([  1.000,   1.000,   1.000])

  return r, th_r, ep_r, mu_r, k0

def S_ProbeFedDLL(k0r, Ang1, Ang2):
  frq = 5*10**9
  c0  = 299792458
  k0  = 2*pi*frq/c0

  L  = 6
  N  = L + 1
  tx = 0.75

  i_   = array(range(N-1,0,-1))
  r__i = list(i_/(N-1))
  th_i = list(zeros(N-2))
  ep_i = list(2 - ((2*i_-1)/(2*(N-1)))**2)
  mu_i = list(ones(N-1))

  r    = diag([       1.000] + r__i)*k0r/k0
  th_r = diag([ Ang1,  Ang2] + th_i)*pi/180
  ep_r = diag([1.000, 1.000] + ep_i)       
  mu_r = diag([1.000, 1.000] + mu_i)       

  return r, th_r, ep_r, mu_r, k0



In [278]:
k0r = 9.95; n = 5; p = "cart"; Lens = "B"; Ang1 = 18; Ang2 = 180; src = 1; c0  = 299792458; et_0 = 4*pi*10**(-7)*c0;

if Lens == "A":
  [r, th_r, ep_r, mu_r, k__0] = S_ProbeFedA(k0r,Ang1,Ang2)
elif Lens == "B":
  [r, th_r, ep_r, mu_r, k__0] = S_ProbeFedB()
else: #Lens == "C"
  [r, th_r, ep_r, mu_r, k__0] = S_ProbeFedDLL(k0r,Ang1,Ang2)

N = len(r)

k_i0 = k__0*sqrt(mult(mu_r[0:N  ,0:N  ],ep_r[0:N  ,0:N  ]));  et_i0 = et_0*sqrt(mult(mu_r[0:N  ,0:N  ],inv(ep_r[0:N  ,0:N  ])))
k_i1 = k__0*sqrt(mult(mu_r[1:N+1,1:N+1],ep_r[1:N+1,1:N+1]));  et_i1 = et_0*sqrt(mult(mu_r[1:N+1,1:N+1],inv(ep_r[1:N+1,1:N+1])))

Ii = diag(ones(N-1),-1); di0 = array(list(map(int, src == array(range(0,N  )))))
Is = diag(ones(N-1), 1); di1 = array(list(map(int, src == array(range(1,N+1)))))

fk = ones(n); fz = 1/(2*array(range(1,n+1))+1); Omg = - inv(r*2*(mult(k_i0,inv(et_i0)) + mult(k_i1,inv(et_i1))))

[Z1_i0,Z3_i0,K1_i0,K3_i0] = F_SphericalFunction(mult(k_i0,r),n);
[Z1_i1,Z3_i1,K1_i1,K3_i1] = F_SphericalFunction(mult(k_i1,r),n);